# Understanding the Functional Testing Approach for PyMBE Playbooks

This notebook walks through the functional testing of interpretation of SysML v2 models within its playbooks. The playbook modules and the graph projections upon which they rely are explored here and connected to bodies of tests.

In [ ]:
from pathlib import Path
import networkx as nx
import matplotlib as plt

import pymbe.api as pm

from pymbe.client import SysML2Client
from pymbe.graph.lpg import SysML2LabeledPropertyGraph
from pymbe.interpretation.interpretation import repack_instance_dictionaries
from pymbe.interpretation.interp_playbooks import (
    build_expression_sequence_templates,
    build_banded_sequence_templates,
    build_sequence_templates,
    random_generator_playbook,
    random_generator_playbook_phase_3_new_instances,
    random_generator_playbook_phase_3_rollup,
)
from pymbe.interpretation.results import *
from pymbe.label import get_label_for_id
from pymbe.query.metamodel_navigator import feature_multiplicity
from pymbe.query.query import (
    roll_up_multiplicity,
    roll_up_upper_multiplicity,
    roll_up_multiplicity_for_type,
    get_types_for_feature,
    get_features_typed_by_type,
)
from pymbe.local.stablization import build_stable_id_lookups

## Gather Files and Load up Test Data

### Load Up Model Circuits File

Read the model from the local JSON file for the Circuits example model and its test information.

In [ ]:
client1 = SysML2Client()

model_file = Path(pm.__file__).parent / "../../tests/fixtures/Circuit Builder.json"

client1._load_from_file(model_file)

circuit_lpg = SysML2LabeledPropertyGraph()
circuit_lpg.model = client1.model
circuit_model = circuit_lpg.model

[circuit_id_to_parts_name_lookup, circuit_parts_name_to_id_lookup] = build_stable_id_lookups(circuit_lpg)

circuit_lpg.model.MAX_MULTIPLICITY = 10

### Load Up Simple Parts Model File

Read the model from the local JSON file for the Simple Parts Model example model and its test information.

In [ ]:
client2 = SysML2Client()

parts_file = Path(pm.__file__).parent / "../../tests/fixtures/Simple Parts Model Banded.json"

client2._load_from_file(parts_file)

simple_parts_lpg = SysML2LabeledPropertyGraph()
simple_parts_lpg.model = client2.model
simple_parts_model = simple_parts_lpg.model

[simple_parts_id_to_parts_name_lookup, simple_parts_name_to_id_lookup] = build_stable_id_lookups(simple_parts_lpg)

simple_parts_lpg.model.MAX_MULTIPLICITY = 10

## Explore Models

This section of the notebook walks through the content of the various models being examined here to help the reader understand how model filters and interpretations should operate upon them.

### Circuits Model

The circuits model is a very simple model intended to explore generative design techniques with a SysML model and then estimate circuit characteristics in OpenMDAO.

In the current iteration of prototyping, the circuits model comes with imported library packages, as seen below.

In [ ]:
circuit_model.packages

The main port, part, and connection definitions are under the main packages.

In [ ]:
circuit_model.ownedElement["Circuit Builder"].ownedElement

The circuit has multiple part and connection usages.

In [ ]:
circuit_model.ownedElement["Circuit Builder"].ownedElement["Circuit"].ownedElement

### Simple Parts Model

In [ ]:
simple_parts_model

In [ ]:
simple_parts_model.packages

In [ ]:
simple_parts_model.ownedElement["Simple Parts Model Banded"].ownedElement

## Interpretation Pre-Work

There are a few steps in the process of generating M0 instances (AKA executions) to make the work easier.

### Feature Sequencing

The M1 model can be examined to set up templates for minimum-length sequences for different sets of nested features.

#### Circuit Model

In [ ]:
filtered_feat_packages = [circuit_model.ownedElement["Circuit Builder"]]

circuit_feature_sequences = build_banded_sequence_templates(lpg=circuit_lpg, package_filter=filtered_feat_packages)

[[circuit_model.elements[typ_id] for typ_id in seq] for seq in circuit_feature_sequences]

#### Simple Parts Banded

In [ ]:
filtered_feat_packages = [simple_parts_model.ownedElement["Simple Parts Model Banded"]]

simple_part_feature_sequences = build_banded_sequence_templates(lpg=simple_parts_lpg, package_filter=filtered_feat_packages)

[[simple_parts_model.elements[typ_id] for typ_id in seq] for seq in simple_part_feature_sequences]

### Feature Multiplicities

When we have the feature sequences worked out, we can then look at the multiplicities at each step.

## Random Interpretation Playbook

These next steps will work through the random interpretation playbook.

### Feature buildup phase

The first phase looks for concrete features from the pre-computed sequences to make instances for.

In [ ]:
starting_circuits_instances = {}
starting_simple_parts_instances = {}

In [ ]:
random_generator_playbook_phase_3_new_instances(circuit_model, circuit_feature_sequences, starting_circuits_instances)
pprint_interpretation(starting_circuits_instances, circuit_model)

In [ ]:
random_generator_playbook_phase_3_new_instances(simple_parts_model, simple_part_feature_sequences, starting_simple_parts_instances)
pprint_interpretation(starting_simple_parts_instances, simple_parts_model)

### Feature rollup phase

With the more specific feature sequences created, use subsetting, redefinition, subclassification, and feature typing

In [ ]:
random_generator_playbook_phase_3_rollup(
    circuit_model,
    circuit_lpg.get_projection("Redefinition and Subsetting"),
    starting_circuits_instances
)
random_generator_playbook_phase_3_rollup(
    circuit_model,
    circuit_lpg.get_projection("Generalization"),
    starting_circuits_instances
)
pprint_interpretation(starting_circuits_instances, circuit_model, show_empty=False)

In [ ]:
random_generator_playbook_phase_3_rollup(
    simple_parts_model,
    simple_parts_lpg.get_projection("Redefinition and Subsetting"),
    starting_simple_parts_instances
)
random_generator_playbook_phase_3_rollup(
    simple_parts_model,
    simple_parts_lpg.get_projection("Generalization"),
    starting_simple_parts_instances
)
pprint_interpretation(starting_simple_parts_instances, simple_parts_model, False)

In [ ]:
m0_interpretation = random_generator_playbook(
    lpg=circuit_lpg,
    name_hints={},
    filtered_feat_packages=[circuit_lpg.model.ownedElement["Circuit Builder"]],
    phase_limit=10
)

In [ ]:
pprint_interpretation(m0_interpretation, circuit_model, False)